In [13]:
# 3

import requests
from bs4 import BeautifulSoup
import pandas as pd

# 페이지 번호를 변경하면서 각 페이지의 URL을 생성하는 함수
# 기사 본문 http 참고함, 008/이후 번호가 datetime에 따라 달라짐
# f스트링으로 {page}번호 변경할 수 있게 함
def generate_url(page):
    return f"https://n.news.naver.com/mnews/article/008/0004944977?sid=105&page={page}"

# 기사 스크래핑 함수
# 기사 링크 받아서 category, title, press, link, document, date 를 dic으로 출력
def art_crawl(url):
    
    art_dic = {}
    
    ## 1.
    category = "모바일" # category는 지정할 수 있기 때문에 모바일로 한정
    
    # url요청 보낼 때 http에러 막기 위해 header 설정함(검색)
    html = requests.get(url, headers={"User-Agent": "Mozilla/5.0"\
    "(Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) "\
    "Chrome/110.0.0.0 Safari/537.36"})
    soup = BeautifulSoup(html.text, "lxml")
    
    ## 2. http를 기사 본문에서 모두 확인하였음
    # 제목 수집
    title = soup.find("h2", class_="media_end_head_headline")
    title_str = title.text.strip() if title else ""
    
    # 언론사 수집
    press = soup.find("em", class_="media_end_linked_more_point")
    press_str = press.text.strip() if press else ""
    
    # 날짜 수집
    date = soup.find("span", class_="media_end_head_info_datestamp_time")
    date_str = date.text.strip() if date else ""
    
    # 본문 수집
    main = soup.find("div", class_="newsct_article")
    main_text = main.text.strip() if main else ""
    
    ## 3.dic에 저장하기
    art_dic["category"] = category
    art_dic["title"] = title_str
    art_dic["press"] = press_str
    art_dic["link"] = url
    art_dic["document"] = main_text
    art_dic["date"] = date_str
    
    return art_dic

# 모든 링크 수집 및 스크래핑
all_hrefs = []
for page in range(1, 101):  # 1부터 100페이지까지 스크래핑
    url = generate_url(page)
    art_dic = art_crawl(url)
    all_hrefs.append(art_dic)

# 데이터프레임 생성
art_df = pd.DataFrame(all_hrefs)

# CSV 파일로 저장
art_df.to_csv("naver_article_df.csv", index=False)
